<a href="https://colab.research.google.com/github/Echo9k/WebScrapping/blob/main/WebScrapping_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
#@title Install libraries
#@markdown use only in colab [Dismissed temporarly]
!rm sample_data -r
!pip install fake-useragent
# !pip install w3lib
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
#@title Set up
#@markdown Loading dependencies...
import os
import re
import scipy
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# HTML
import requests
from bs4 import BeautifulSoup as bs
from requests.exceptions import HTTPError
from IPython.core.display import display, HTML
from urllib.parse import unquote
# import mechanize

# Selenium for JS support
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
# from selenium import webdriver

In [ ]:
#@title Headless
#@markdown As: headles_driver
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# #@markdown As: headles_driver
# headles_driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
#@title PhantomJS
#@markdown As: phantom_driver
# from selenium import webdriver
# !wget https://bitbucket.org/ariya/phantomjs/downloads/phantomjs-2.1.1-linux-x86_64.tar.bz2
# !tar xvjf phantomjs-2.1.1-linux-x86_64.tar.bz2
# !cp phantomjs-2.1.1-linux-x86_64/bin/phantomjs /usr/local/bin
# !ls -al
# phantom_driver = webdriver.PhantomJS()

# Explore with tags
One option we have to obtain the product detaill's from a URL would be using the labels corresponding and attribute to find their necesary value. We can use Regular Expressions (RegEx) to speed up this process.

In [ ]:
#@markdown Regex strings defined under: *variable*_regex
brand_regex = r"(?:brand|brandname|vendor|manufacturer|product-brand)(?![&])(.)"
crumb_regex = r"(?:category|categories|category path|breadcrumbs|breadcrum|crumb|navbar|Product Category)(?![&])(.)"
sku_regex = r"(?:sku|model|model id|model no|item number|itemid|article no|product number|style number|product id|item code|mfr no|data-product)(?![&])(.)"
model_regex = '(?:sku|model|model id|model no|item number|itemid|article no|product number|style number|product id|item code|mfr no|data-product)(?![&])(.)'
upc_regex = r'(?:"UPC"|"GTIN"|"EAN"|"upc"|"upccode"|"product_upc"|"product:upc"|"gtin"|"ean"|"barcode")'
part_regex = r"(?:PN|P/N|part no|part number|part|part #|mpn)(?![&g])(...)"
color_regex = r"(?:color|color_name|shade|finish|shade description)(?![&])(.)"
size_regex = r"(?:selected size|available size|choose a size|product size|attribute pa size)(?![&])(.)"
mfr_regex = r"(?:manufacturer|mfr|mfg|manufacturer logo|manufacturer name|label|producer|fabricante|fabrikant|Hersteller)(?![&])(.)"
price_regex = r"(?:MSRP|MRP|Recommended Customer Price|USD MSRP|List Price|reseller price may vary)(?![&])(.)"
ct_regex = r"(?:count|pieces|ct|pc|combo|per pack|contains)(?![&])(.)"
pk_regex = r"(?:packs|packs of|pk|package|combo|carton|carton pack)(?![&])(.)"
description_regex = r"(?:Product Details|Specification|Tech specs|Technical specification|Details|see more features|Product Description|Description|About the product|ingredients|Where to use|How to use)(?![&])(.)"
#@markdown The RegEx strings are stored in the directory: _labels_ <br> <br>
labels= {
    "Brand Name":brand_regex,
    "Category Name":crumb_regex,
    "SKU":sku_regex,
    "Model Name":model_regex,
    "UPC":upc_regex,
    "Part Number":part_regex,
    "Color name":color_regex,
    "Size Name":size_regex,
    "Manufacturer Name":mfr_regex,
    "List Price":price_regex,
    "Item Count":ct_regex,
    "Item Package Quantity":pk_regex,
    "Product Description":description_regex
    }

#@markdown ### Functions
#@markdown * Search tag
#@markdown * Finder
def search_tag(tag, string):
    regex=r"(?:"+tag+"=)"
    if tag in string:
        split_1 = re.split(regex,string)[1].replace('%20', ' ')
        print(f"{tag}: found in text\n"
            f"contains:{split_1}")

def finder(regex:str, text:str,*,
           look_before:int=10,
           look_ahead:int=250,
           extra_dots=1) -> str:
    """
    # RETURNS: group found, match
    """
    matches = re.finditer(regex, text, re.MULTILINE | re.IGNORECASE | re.UNICODE)
    
    for matchNum, match in enumerate(matches, start=1):
        print(f"Match: {matchNum} {match.group()}")
        # , match = match.group()
        
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            return match.group()[:-extra_dots], match.string[match.start(groupNum)-look_before:match.end(groupNum)+look_ahead]

In [ ]:
#@title Read URL
# Functions
render = lambda html_contents: display(HTML(html_contents))

# Attributes
web_driver = False #@param {type:"boolean"}
URL = "http://www.kregtool.com/on/demandware.store/Sites-kreg-Site/en_US/Search-ShowAjax?cgid=shop&prefn1=badges&prefv1=awardWinning&prefn2=type&prefv2=Bench%20Clamps%20with%20Base%7cClamp%20Vises%7cTrack%20Clamp%7cClamp%20Table%7cDriver%20Bits%7cHardware%20Containers%7cProject%20Table%20Replacement%20Top%7cSwing%20Stop%7cRouter%20Table%20Fence%7cPlug%20Cutters%7cBeading%20Bit%7cScrew%20Kits%7cRouter%20Table%20Setup%20Bars%7cJig%20%26%20Fixture%20Bar%7cNotching%20Bit%7cDeck%20Screws%7cClamp%20Trak%20Kits%7cReplacement%20Guide%20Strips%7cCrown%20Molding%20Cutter%7cScrews%7cStepped%20Drill%20Bits%7cRail%20Sets%7cRouter%20Table%20Switch%7cVersa%20Stops%7cClamp%20Plates%7cClamp%20Traks%7cJackets%7cMiter%20Gauge%7cJig%20Expansion%20Pack%7cGuide%20Blocks%7cDeck%20Jig%7cMeasuring%20Tool%7cProject%20Table%20Tops%7cExpansion%20Pack%7cProject%20Table%20Kit%7cRight%20Angle%20Clamps%7cRouter%20Lift%7cDrawer%20Slide%20Jigs%7cPocket-Hole%20Machines%7cPlugs%7cCabinet%20Hardware%20Jigs%7cMaster%20Kit%7cDriver%20Combo%7cFace%20Clamps%7cClamp%20Table%20and%20Stand%20Combo%7cDrill%20Guide%20Spacer%20Block%7cTrack%20Horses%7cCombo%20Track%7cSaw%20%26%20Guide%20Track%20Kit%7cHats%7cPlunge%20Saw%7cBand%20Saw%20Fence%7cDust%20Collection%20Attachment%7cMicro%20Adjuster%7cPlate%20Levelers%7cFace-Framing%20Table%7cRouter%20Table%20Top%7cRouter%20Table%20System%7cHinge%20Jigs%7cSaw%20Track%20Combo%7cAdhesive%20Measuring%20Tape%7cDrill%20Bits%7cSystem%20Organizer%7cRip%20Guides" #@param {type:"string"}
show = False #@param {type:"boolean"}

# Retrieve URL
if len(URL)>0:
    response = requests.request('GET', URL)
    soup = bs(response.text)
    pretty_soup = soup.prettify()

    if web_driver:
        wd.get(URL)

    if show:
        render(response.text)

In [ ]:
try:
    find = "Manufacturer Name" #@param ['Brand Name', 'Category Name', 'Model Name', 'UPC', 'Part Number', 'Color name', 'Size Name', 'Manufacturer Name', 'List Price', 'Item Count', 'Item Package Quantity', 'Product Description']
    finder(labels.get(find), soup.text, look_ahead=10)
except:
    "something happened"

In [ ]:
#@title By id and attribute
try:
    id = "ProductPrice" #@param {type:"string"}
    attribute = "itemprop" #@param {type:"string"}
    try:
        wd.find_element_by_id(id).get_attribute(attribute=attribute)
    except:
        print("something happened")
except:
    "something happened"

Most of this can be done through the libraries: soup, and BeautifulSoup4.

For sites which rely heavly on JS it can also be usefull to use Phantom or headless_driver to access specific attributes.
```
def get_attribute(id, attribute):
    return headles_driver.find_element_by_id(id).get_attribute(attribute)
```
These two last options are slower to load and often you can find a workaround using soup.decode.




# Recursive search
A different paradigm is to use a list of URL to retrive and save their soups at once, thus reducing server work and reducing processing time.

## Additional setup

In [ ]:
#@title Additional libraries

#@markdown * Import libraries
import json
import time
import unicodedata
from fake_useragent import UserAgent
# from w3lib.html import replace_entities
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from google.colab import data_table

In [ ]:
#@title Introducing a Class: RetrivePage

class RetrivePage:
    "This is a page class meant to retrive a page and return it's soup object"
    def __init__(self, url, headless=False, phantom=False):
        __variant_extractor_lambda = lambda x: np.squeeze(re.findall(r"=(.*)", x))
        self.url = url
        self.soup = bs(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text)
        self.parsed_url = unquote(url)
        self.headless_driver = None
        self.phantom_driver = None
        self.variant = __variant_extractor_lambda(self.url)
        self.metadata = {}
        self.variant_info = {}
        if headless:
            self.headless_driver = get_headless_driver()
        if phantom:
            self.phantom_driver = get_phantom_driver()
    
    def apply(self, function):
        return function(self.page)
    
    # # Headless driver
    # def get_headless_driver(self):
    #     return headles_driver.get(self.url)
    # def build_headless(self):
    #     self.headless_driver = self.get_headless_driver()

    # # Phantom driver
    # def build_phantom(self):
    #     self.phantom_driver = self.get_phantom_driver()
    # def get_phantom_driver(self):
    #     return phantom_driver.get(self.url)

In [ ]:
#@title Functions
#@markdown * Retriving data
#Functions
def load_file(file_name, usecols):
    def __read_csv(file_name, usecols=useful_cols, **kwargs):
        try:
            return pd.read_csv(file_name, usecols)
        except:
            display("Dismissed: useful_cols")
            return pd.read_csv(file_name)

    def __read_excel(file_name, usecols=useful_cols, **kwargs):
        try:
            return pd.read_csv(file_name, usecols)
        except:
            display("Dismissed: useful_cols")

            return pd.read_excel(file_name)
    # read
    if file_name.rsplit('.',1)[-1] in ['csv','text']:
        print('text')
        return __read_csv(file_name, usecols=useful_cols, **kwargs)
    elif file_name.rsplit('.',1)[-1] in ['xls','xlsx']:
        print('Excel')
        return __read_excel(file_name, usecols=useful_cols, **kwargs)
    else:
        "unknown format"
#@markdown List of useful columns
#["url","brand_name","manufacturer_name","product_description","model","color_name","item_package_quantity", "unit_count"]
all_columns=["title","bread_crumb1","bread_crumb2","bread_crumb3","brand_name",
             "manufacturer_name","model","upc","color_name","size_name",
             "item_package_quantity","part_number","list_price","unit_count",
             "product_description"]
useful_cols =  ["url","brand_name","manufacturer_name","product_description","model","color_name","item_package_quantity", "unit_count"]#@param {type:"raw"}

## Workspace

In [ ]:
#@title Get the Websites
wait_time = 0 #@param {type:"slider", min:0, max:10, step:0.1}
#@markdown Select **user agent**
# User agent to workaround sites that don't allow bots
header = "Chrome" #@param ["Chrome", "Edge", "Random", "None"]
usr_agent = UserAgent()

def user_agent_retrive(i):
    switcher={
            "Edge":usr_agent.edge,
            "Chrome":usr_agent.chrome,
            "Random":usr_agent.random
            }
    return switcher.get(i,None)
header = user_agent_retrive(header)
#@markdown _Looking for more options?_  [fake_useragent](https://pypi.python.org/pypi/fake-useragent)
def load_URLs(URL):
    r = requests.request('GET', URL, headers={'User-Agent': usr_agent})
    time.sleep(wait_time)
    return bs(r.text), phantom_driver.get(URL), headles_driver.get(URL)
file_name = "/content/fadlash.com_products.csv" #@param {type:"string"}
kwargs  =  {} #@param {type:"raw"}

#@markdown Data loaded as: data
data = load_file(file_name, usecols=useful_cols)

## separating URLs
url_variant = data.url[data.url.apply(lambda x: "=" in x)]
not_variant = data.url[data.url.apply(lambda x: "=" not in x)]

print(f"Not variant URLs: {len(not_variant)}\n"
      f"URL with variant: {len(url_variant)}")

In [ ]:
#@markdown Create a new pd.Series to store the web scrapped RetrivePage objects.<br>
#@markdown ...loading
start_time = time.time()
stopwatch = lambda x: time.time() - start_time

pages_1 = data[:40].url.apply(RetrivePage), print("20%\t", f"time {stopwatch(start_time)}")
pages_2 = data[40:80].url.apply(RetrivePage), print("40%\t", f"time {stopwatch(start_time)}")
pages_3 = data[80:120].url.apply(RetrivePage), print("60%\t", f"time {stopwatch(start_time)}")
pages_4 = data[120:160].url.apply(RetrivePage), print("80%\t", f"time {stopwatch(start_time)}")
pages_5 = data[160:].url.apply(RetrivePage), print("100%\t", f"time {stopwatch(start_time)}")
pages = pd.concat([pages_1[0], pages_2[0], pages_3[0],pages_4[0], pages_5[0]])
print("Completed%", "time: ", stopwatch(start_time))

In [ ]:
#@markdown ### **Start** ► URL & title
_get_url = lambda x:x.url
urls = pages.apply(_get_url)
result = {"url":urls} #@param {type:"raw"}
#@markdown include titles?
boolean = False #@param {'type':'boolean'}
if boolean:
    _get_title = lambda x:x.soup.title.text.strip('\n').rstrip('\n')
    titles = pages.apply(_get_title)
    result.update({"title":titles})

In [ ]:
#@markdown Progress
show_progress = True #@param {type:"boolean"}
df_result = pd.DataFrame(result)
df_result.fillna("___BLANK_CELL___", inplace=True)
if show_progress:
    display(data_table.DataTable(df_result))

In [ ]:
df_result[0:50].replace('',fill_na_with)

In [ ]:
#@markdown Export
reindex_join_df = True  #@param {type:"boolean"}
fill_na_with = "___BLANK_CELL___" #@param  {type:"string"}
rsuffix='_reviewed' #@param {type:"string"}

_export_name = file_name.split('.')[0] + "_reviwed.xlsx"
_result_column_set = set(df_result.columns)
_data_column_set = set(data.columns)
_intersection = _data_column_set.intersection(_result_column_set)
_intersection.discard('url') # inplace opperation
print("Intersection", _intersection)

df_join = data.drop(columns=_intersection)
df_join = pd.concat([df_join.set_index('url'), df_result.set_index('url')],axis=1,)\
                .fillna(fill_na_with)

_df_join_columns=set(df_join.columns)
_df_join_columns.discard('url')

if reindex_join_df:
    try:
        df_join = df_join.reindex(columns=list(_df_join_columns))
        print(f"df_join \t[reindexed]\t{time.time()}")
    except:
        print(f"df_join \t[couldn't reindex]\t{time.time()}")
        display(_df_join_columns.symmetric_difference(_data_column_set))
    df_join.to_excel(_export_name)
else:
    df_result.to_excel(_export_name)
    print(f"df_result\t{time.time()}")

In [ ]:
# df_result = df_result.rename(columns={'model':barcode,"upc":upcs, "notes":size_})

In [ ]:
#@markdown Find url' index
urls_dict = {url:i for i,url in enumerate(urls)}
url = "https://fadlash.com/products/eyelash-extension-glue-sky-s?variant=32657254154319" #@param {type:"string"}
urls_dict.get(url)

In [ ]:
#@markdown Example page: example
index =    144#@param {type:"integer"}
page = pages[index]
soup = page.soup
url = page.url
variant = page.variant
example = {'index':index,
           'page':page,
           'soup':soup,
           'url':url,
           'variant':variant}
print(f"url: {page.url}")
show = False #@param {type:"boolean"}
if show:
    render(response.text)

## Finding additional attributes

#### From scripts
Often pages relay on a data structure for keeping the information of the different variants.

In [ ]:
#@markdown For when the metadata is not in a regular variable
identifier="OptionSelectors('productSelect', {\n      product: "

def __get_metadata(page):
    script = [str(i) for i in soup.find_all('script') if identifier in str(i)][0]
    script = script.split(identifier)[1]
    script = script.split("onVariantSelected:")[0].rstrip('\n,\n      ')
    metadata = json.loads(script)
    page.metadata = metadata
    # return colors 

metadata = pages.apply(__get_metadata)

In [ ]:
# @title Metadata
# @markdown This is useful when you have a tag such as **var meta =**
var_name = "var meta = " #@param {'type':'string'}
key_name = "meta" #@param {'type':'string'}
if key_name is None:
    key_name = var_name.strip().split()[1]
get_variant_info = True #@param {type:"boolean"}

print(f"key_name: {key_name}")
def __get_variant_info(page):
    # try:
    #     variants = page.metadata['product']['variants']
    #     return [i for i in variants if int(i['id']) == int(page.variant)][0]
    # except: None
    try: return [v for v in page.metadata[key_name]['product']['variants'] if int(v['id']) == int(page.variant)][0]
    except: None

def get_meta(page):
    soup = page.soup
    scripts = soup.find_all('script')
    # mets = ''
    for s in scripts:
        if var_name in s.text:          # find the script of interest
            # Exctract the json value of var_name
            meta = s.text
            meta = meta.split(var_name)[1].split('};')[0]+'}'
            # Load the json and make's sure the format is correct
            json_meta = json.loads(meta)
            page.metadata.update({key_name:json_meta}) # add metadata to the RetrivedPage
            if get_variant_info:
                page.variant_info.update({key_name:__get_variant_info(page)})
            return json_meta
json_meta = pages.apply(get_meta)
# @markdown **json_meta*: results
# result_key = "metadata" #@param {type:"string"}
# result.update({result_key:metadata}) 

In [ ]:
#@markdown Variant attributes
def _extract_options(page):
    try:
        return {
            "name":page.variant_info['meta']['name'],
            # "barcode":page.variant_info['SUBParams']['barcode'],
            "sku":page.variant_info['meta']['sku'],
            'public_title':page.variant_info['meta']['public_title']
            }
    except: return {"name":None,"barcode": None,"sku":None, "public_title":None}
ids = pages.apply(_extract_options)

names = ids.apply(lambda x: x['name'])
# barcode = ids.apply(lambda x: x['barcode'])
upcs = ids.apply(lambda x: x['sku'])
sizes = ids.apply(lambda x: x['public_title'])

In [ ]:
#@markdown experiments
def __get_options(page):
    try: 
        options = [i for i in page.variant_info['name'].split() if i != '-'][:2]
    except:
        options = [None,None]
    return {"color":options[0], "size":options[1]}
options = pages.apply(__get_options)
size_ = options.apply(lambda x: x['size'])
color = options.apply(lambda x: x['color'])

In [ ]:
result.update({"name":names,
               "upc":upcs})

### Done:

In [ ]:
result.update({"title":names,
               "bread_crumb1":bread_crumb1,
               "bread_crumb2":bread_crumb2,
               "bread_crumb3":bread_crumb3,
            #    "product_description":descriptions,
            #    "color_name":colors,
            "size_":size_,
               "size_name":sizes,
               "upc":upcs,
               "barcode":barcode})

In [ ]:
#@markdown Title
def __get_titles(page):
    try:
        page.title = page.soup.h1.text
        return page.soup.h1.text
    except: return None

titles=pages.apply(__get_titles)
titles

In [ ]:
#@markdown Crumbs
def __get_crumbs(page):
    try: 
        crumbs = page.soup\
                    .find("div", {'id':"breadcrumb",
                                  "class":"desktop-12 tablet-6 mobile-3"})\
                    .text.strip().split('\n»\n')
    except:
        return (None, None, None)
    try:
        if page.title != crumbs[2]:
            return (crumbs[0], crumbs[1], crumbs[2])
        else:
            return (crumbs[0], crumbs[1], None)
    except:
        try:
            return (crumbs[0], crumbs[1], None)
        except:
            return (None, None, None)
bread_crumbs = pages.apply(__get_crumbs)
bread_crumb1 = crumbs.apply(lambda x: x[0])
bread_crumb2 = crumbs.apply(lambda x: x[1])
bread_crumb3 = crumbs.apply(lambda x: x[2])

In [ ]:
#@markdown Descriptions
def __get_descriptions(page):
    try: return unicodedata.normalize("NFKC", page.soup\
                    .find("div", {"class":"product-description rte",
                                "itemprop":"description"})\
                    .text.strip('\n'))
    except: return None
descriptions = pages.apply(__get_descriptions)

In [ ]:
#@markdown UPC
# pattern=re.compile('[\w\d-]+')

def __get_upc(page):
    try: return page.variant_info['sku']
    except: return None

upcs = pages.apply(__get_upc)

### Work In Progress

#### Experiments

In [ ]:
variant_info = pages.apply(__variant_info)
def __get_barcode(b):
    try: return str(b['barcode'])
    except: return b
barcodes = barcode.apply(__get_barcode)

In [ ]:
#@title Description
def __get_descriptions(page):
    soup = page.soup
    content = soup.find_all('div', {"class":"TabbedPanelsContent"})
    description_tabs = soup.find("ul", attrs={"class":"TabbedPanelsTabGroup"})
    description_tabs = description_tabs.text\
                                    .rstrip('\n').strip('\n')\
                                    .split('\n')

    description = ''
    ignore = ['Optional Accessories', 'Impeller Kits']

    def clean_text(text_to_clean):
        return text_to_clean.strip('\n').rstrip('\n').strip('\xa0').rstrip('\n').strip('\n')

    def __post_process_description(description):
        description_noWarranty = ''
        for i, strr in enumerate(description.split('\n')):
            if (len(strr) < 15) & ("arranty" in strr):
                ++i
            else:
                description_noWarranty += strr
        return description_noWarranty\
                            .rstrip('\n')\
                            .replace('â\x80¢', '•')\
                            .replace('Â°', '°')\
                            .replace('âs',"'s")
                            

    try:
        for i, tab_name in enumerate(description_tabs):
            if tab_name not in ignore:
                info = clean_text(content[i].text)
                description += tab_name +  '\n' + info + '\n'
        return __post_process_description(description)
    except IndexError:
        print(page.url)
descriptions = pages.apply(__get_descriptions)
# df = df.join(descriptions)

### json

In [ ]:
#@markdown Find Attributes
dc = json.loads(finder(r'(?:var meta = )(.*)(?:};)', s.text)[0].replace('var meta = ',''))
try:
    variant_num = URL.split('variant=')[1]
    variant = [i for i in dc['product']['variants'] if i['id']==int(variant_num)][0]
except IndexError:
    variant = dc['product']['variants'][0]
#@markdown * title
title = driver.find_element_by_class_name("standard-single").text

#@markdown * Public_title  | _has the size values_
public_title = variant['public_title'] 

#@markdown * brand
brand = finder(r"(?:\"brand\")(.)", s.text, look_ahead=20)[1].split(":")[1].replace('"','')

#@markdown * manofacturer
try:
    label = dc['product']['label']
except:
    label = finder(r"(?:\"label\")(.)", s.text, look_ahead=20)[1].split(":")[1].replace('"','')

#@markdown * Price
price = wd.find_element_by_id('ProductPrice')

# Crawler
Finally, another paradigm is defining a the page structure to obtain the values using a crawler.

In [ ]:
#@title Imports
import requests
from bs4 import BeautifulSoup

In [ ]:
# Key- Retrival function
kwargs = {'title':(lambda p: p.title)}

# Class: Website 
class Website:
    """
    Contains information about website structure.
    """
    def __init__(self, url, **kwargs):
        self.url = url
        self.__dict__.update(kwargs)

# Initialize the website as an empty entity
# kwargs_n = {keys:None for keys in kwargs.keys()}
# w = Website(page.url, kwargs_n)

In [ ]:
class Crawler:
    def __init__(self, attrs:[dir]):
        self.attrs=attrs

    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        """
        Utility function used to get a content string from a
        Beautiful Soup object and a selector. Returns an empty
        string if no object is found for the given selector
        """
        selectedElems = pageObj.soup.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text()
                for elem in selectedElems])
        return 'empty'

    def ifer(self, page, sfun:[str, callable]):
        if callable(sfun):
            return sfun(page)
        else:
            return self.safeGet(page, sfun)

    def parser(self, pageObj=None, url:[str]=None):
        """
        Extract content from a given page URL
        """
        if pageObj is None:
            if url is not None:
                pageObj = self.getPage(url)
            else:
                "You need to pass one of pageObj/url"
        else:
            url = pageObj.url

        attrs = {key:self.ifer(pageObj, sfun)
                for key, sfun in self.attrs.items()
                }
        return Website(pageObj.url, page=pageObj, **attrs)

In [ ]:
# c = Crawler(kwargs)
w=c.parser(page.soup, page.url)
page.url